In [17]:
from transformers import pipeline, set_seed
from datasets import load_from_disk
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch


In [3]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch 
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [31]:
from pathlib import Path
PATH_DATA_BASE = Path.cwd() / "data" / "sumsum_dataset"
# List files in the directory
file_list = [file.name for file in PATH_DATA_BASE.iterdir() if file.is_file()]

print("Files in the directory:")
for filename in file_list:
    print(filename)

Files in the directory:
samsum-test.csv
samsum-train.csv
samsum-validation.csv


In [4]:
model_ckpt = "google/pegasus-cnn_dailymail"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
dataset_samsum = load_from_disk(PATH_DATA_BASE)


FileNotFoundError: Directory C:\Users\soulo\MACHINE_LEARNING\PaperMate\data\sumsum_dataset is neither a `Dataset` directory nor a `DatasetDict` directory.

In [6]:
dataset_samsum_train = pd.read_csv('data/sumsum_dataset/samsum-test.csv')
dataset_samsum_test= pd.read_csv('data/sumsum_dataset/samsum-train.csv')
dataset_samsum_validate= pd.read_csv('data/sumsum_dataset/samsum-validation.csv')

In [11]:
print(f"Features: {dataset_samsum_train.columns}")
print("\nDialogue:")

print(dataset_samsum_test[:5]["dialogue"])

print("\nSummary:")

print(dataset_samsum_test[:5]["summary"])

Features: Index(['id', 'dialogue', 'summary'], dtype='object')

Dialogue:
0    Amanda: I baked  cookies. Do you want some?\r\...
1    Olivia: Who are you voting for in this electio...
2    Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...
3    Edward: Rachel, I think I'm in ove with Bella....
4    Sam: hey  overheard rick say something\r\nSam:...
Name: dialogue, dtype: object

Summary:
0    Amanda baked cookies and will bring Jerry some...
1    Olivia and Olivier are voting for liberals in ...
2    Kim may try the pomodoro technique recommended...
3    Edward thinks he is in love with Bella. Rachel...
4    Sam is confused, because he overheard Rick com...
Name: summary, dtype: object


In [12]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
    
    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )
        
    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }
    

TypeError: convert_examples_to_features() got an unexpected keyword argument 'batched'